In [1]:
HTML(read(open("style.css"), String))

HTML{String}("<link href='http://fonts.googleapis.com/css?family=Alegreya+Sans:100,300,400,500,700,800,900,100italic,300italic,400italic,500italic,700italic,800italic,900italic' rel='stylesheet' type='text/css'>\r\n<link href='http://fonts.googleapis.com/css?family=Arvo:400,700,400italic' rel='stylesheet' type='text/css'>\r\n<link href='http://fonts.googleapis.com/css?family=PT+Mono' rel='stylesheet' type='text/css'>\r\n<link href='http://fonts.googleapis.com/css?family=Shadows+Into+Light' rel='stylesheet' type='text/css'>\r\n<link href='http://fonts.googleapis.com/css?family=Philosopher:400,700,400italic,700italic' rel='stylesheet' type='text/css'>\r\n\r\n<style>\r\n\r\n@font-face {\r\n    font-family: \"Computer Modern\";\r\n    src: url('http://mirrors.ctan.org/fonts/cm-unicode/fonts/otf/cmunss.otf');\r\n}\r\n\r\n.container { width: 100% }\r\n\r\n/* Formatting for header cells */\r\n.text_cell_render h1 {\r\n    font-family: 'Philosopher', sans-serif;\r\n    font-weight: 400;\r\n    font-size: 2.2em;\r\n    line-height: 100%;\r\n    color: rgb(0, 80, 120);\r\n    margin-bottom: 0.1em;\r\n    margin-top: 0.1em;\r\n    display: block;\r\n}\t\r\n.text_cell_render h2 {\r\n    font-family: 'Philosopher', serif;\r\n    font-weight: 400;\r\n    font-size: 1.9em;\r\n    line-height: 100%;\r\n    color: rgb(200,100,0);\r\n    margin-bottom: 0.1em;\r\n    margin-top: 0.1em;\r\n    display: block;\r\n}\t\r\n\r\n.text_cell_render h3 {\r\n    font-family: 'Philosopher', serif;\r\n    margin-top:12px;\r\n    margin-bottom: 3px;\r\n    font-style: italic;\r\n    color: rgb(94,127,192);\r\n}\r\n\r\n.text_cell_render h4 {\r\n    font-family: 'Philosopher', serif;\r\n}\r\n\r\n.text_cell_render h5 {\r\n    font-family: 'Alegreya Sans', sans-serif;\r\n    font-weight: 300;\r\n    font-size: 16pt;\r\n    color: grey;\r\n    font-style: italic;\r\n    margin-bottom: .1em;\r\n    margin-top: 0.1em;\r\n    display: block;\r\n}\r\n\r\n.text_cell_render h6 {\r\n    font-family: 'PT Mono', sans-serif;\r\n    font-weight: 300;\r\n    font-size: 10pt;\r\n    color: grey;\r\n    margin-bottom: 1px;\r\n    margin-top: 1px;\r\n}\r\n\r\n.text_cell_render em {\r\n    font-family: 'Philosopher', sans-serif;\r\n    color:        blue;\r\n    background-color: rgb(255,220,180);\r\n    font-size:    110%;\r\n    margin-left:   2px;\r\n    margin-right:  2px;\r\n    font-weight:   100;\r\n}\r\n\r\n.text_cell_render b {\r\n    color:            rgb(255,195,195);\r\n    background-color: rgb(0,0,0);\r\n    font-size:    110%;\r\n    margin-left:   2px;\r\n    margin-right:  2px;\r\n    font-weight:   650;\r\n}\r\n\r\n.text_cell_render tt {\r\n    font-size:    120%;\r\n    margin-left:   2px;\r\n    margin-right:  2px;\r\n    font-weight:   150;\r\n}\r\n\r\n.Codemirror {\r\n    font-family: \"PT Mono\";\r\n    font-size: 100%;\r\n}\r\n\r\n</style>\r\n\r\n")

# Simple Evaluation Function

This notebook can return a static centipawn value for each chess position. It will not look into the future and only consider the current state of the board. 

This method has been developed by Tomasz Michniewski in the Polish chess programming discussion list (progszach).


https://www.chessprogramming.org/Simplified_Evaluation_Function

In [2]:
using Pkg
#Pkg.add("Chess")
using Chess

In [3]:
pawn_square_table = [
    [0,  0,  0,  0,  0,  0,  0,  0],
    [50, 50, 50, 50, 50, 50, 50, 50],
    [10, 10, 20, 30, 30, 20, 10, 10],
    [5,  5, 10, 25, 25, 10,  5,  5],
    [0,  0,  0, 20, 20,  0,  0,  0],
    [5, -5,-10,  0,  0,-10, -5,  5],
    [5, 10, 10,-20,-20, 10, 10,  5],
    [0,  0,  0,  0,  0,  0,  0,  0]
]


knight_square_table = [
    [-50,-40,-30,-30,-30,-30,-40,-50],
    [-40,-20,  0,  0,  0,  0,-20,-40],
    [-30,  0, 10, 15, 15, 10,  0,-30],
    [-30,  5, 15, 20, 20, 15,  5,-30],
    [-30,  0, 15, 20, 20, 15,  0,-30],
    [-30,  5, 10, 15, 15, 10,  5,-30],
    [-40,-20,  0,  5,  5,  0,-20,-40],
    [-50,-40,-30,-30,-30,-30,-40,-50]
]

bishop_square_table = [
    [-20,-10,-10,-10,-10,-10,-10,-20],
    [-10,  0,  0,  0,  0,  0,  0,-10],
    [-10,  0,  5, 10, 10,  5,  0,-10],
    [-10,  5,  5, 10, 10,  5,  5,-10],
    [-10,  0, 10, 10, 10, 10,  0,-10],
    [-10, 10, 10, 10, 10, 10, 10,-10],
    [-10,  5,  0,  0,  0,  0,  5,-10],
    [-20,-10,-10,-10,-10,-10,-10,-20]
]

rook_square_table = [
    [ 0,  0,  0,  0,  0,  0,  0,  0],
    [ 5, 10, 10, 10, 10, 10, 10,  5],
    [-5,  0,  0,  0,  0,  0,  0, -5],
    [-5,  0,  0,  0,  0,  0,  0, -5],
    [-5,  0,  0,  0,  0,  0,  0, -5],
    [-5,  0,  0,  0,  0,  0,  0, -5],
    [-5,  0,  0,  0,  0,  0,  0, -5],
    [ 0,  0,  0,  5,  5,  0,  0,  0]
]

queen_square_table = [
    [-20,-10,-10, -5, -5,-10,-10,-20],
    [-10,  0,  0,  0,  0,  0,  0,-10],
    [-10,  0,  5,  5,  5,  5,  0,-10],
    [ -5,  0,  5,  5,  5,  5,  0, -5],
    [  0,  0,  5,  5,  5,  5,  0, -5],
    [-10,  5,  5,  5,  5,  5,  0,-10],
    [-10,  0,  5,  0,  0,  0,  0,-10],
    [-20,-10,-10, -5, -5,-10,-10,-20]
]

king_square_middle_game_table = [
    [-30,-40,-40,-50,-50,-40,-40,-30],
    [-30,-40,-40,-50,-50,-40,-40,-30],
    [-30,-40,-40,-50,-50,-40,-40,-30],
    [-30,-40,-40,-50,-50,-40,-40,-30],
    [-20,-30,-30,-40,-40,-30,-30,-20],
    [-10,-20,-20,-20,-20,-20,-20,-10],
    [ 20, 20,  0,  0,  0,  0, 20, 20],
    [ 20, 30, 10,  0,  0, 10, 30, 20]
]

king_square_end_game_table = [
    [-50,-40,-30,-20,-20,-30,-40,-50],
    [-30,-20,-10,  0,  0,-10,-20,-30],
    [-30,-10, 20, 30, 30, 20,-10,-30],
    [-30,-10, 30, 40, 40, 30,-10,-30],
    [-30,-10, 30, 40, 40, 30,-10,-30],
    [-30,-10, 20, 30, 30, 20,-10,-30],
    [-30,-30,  0,  0,  0,  0,-30,-30],
    [-50,-30,-30,-30,-30,-30,-30,-50]
]

8-element Vector{Vector{Int64}}:
 [-50, -40, -30, -20, -20, -30, -40, -50]
 [-30, -20, -10, 0, 0, -10, -20, -30]
 [-30, -10, 20, 30, 30, 20, -10, -30]
 [-30, -10, 30, 40, 40, 30, -10, -30]
 [-30, -10, 30, 40, 40, 30, -10, -30]
 [-30, -10, 20, 30, 30, 20, -10, -30]
 [-30, -30, 0, 0, 0, 0, -30, -30]
 [-50, -30, -30, -30, -30, -30, -30, -50]

More efficient solution for get_Intex_by_file

In [4]:
function get_value(piece::Piece)
    values = Dict([(PAWN, 100),(KNIGHT, 320),(BISHOP, 330),(ROOK, 500),(QUEEN, 900),(KING, 20000)])
    pieceType = ptype(piece)
    if pieceType in values.keys
        return values[pieceType]
    end
end

get_value (generic function with 1 method)

According to the article:
"Additionally we should define where the ending begins. For me it might be either if:

Both sides have no queens or
Every side which has a queen has additionally no other pieces or one minorpiece maximum."

In [5]:
function is_endgame(board)
    # Inline Functions to calculate the number of minor pieces
    num_minor_pieces(side) = squarecount(knights(board, side)) + squarecount(bishops(board, side))
    # Check if Player White or Black is in endgame
    is_in_endgame(color) = isempty(queens(board, color)) || (num_minor_pieces(color) <= 1 && isempty(rooks(board, color)))
    return is_in_endgame(WHITE) && is_in_endgame(BLACK)
end

is_endgame (generic function with 1 method)

## Get_Square_Value

The function `get_square_value` is used to evaluate the position of a chess piece on a board. As input parameters it takes a board (board::Board), a chess piece (piece::Piece) and a playing field (square::Square) on which the piece is located. The function calculates a numeric centipawn value that describes the strength or weakness of the current position of the piece. 

The King uses two tables. The `table_opt` can be set to "end" to enforce the usage of the endgame table. This feature overrides the actual position of the board.

In [6]:
function get_square_value(board::Board, piece::Piece, square::Square, table_opt = "middle")
    PIECE_SQUARE_TABLES = Dict(
        PAWN => pawn_square_table,
        KNIGHT => knight_square_table,
        BISHOP => bishop_square_table,
        ROOK => rook_square_table,
        QUEEN => queen_square_table,
        KING => king_square_middle_game_table
    )
    piece_type = ptype(piece)
    if piece_type in PIECE_SQUARE_TABLES.keys
        square_table = PIECE_SQUARE_TABLES[piece_type]
    else
        error("Invalid piece type: $(piece_type)")
    end
    if piece_type == KING && (table_opt == "end" || is_endgame(board))
        square_table = king_square_end_game_table
    end
    squareString = tostring(square)
    x = Int(squareString[1]) - Int('a')+1
    y = parse(Int64, squareString[2]) 
    if Chess.pcolor(piece) == BLACK
        square_table = reverse(square_table)
    end
    return square_table[9 - y][x]
end

get_square_value (generic function with 2 methods)

In [7]:
function calc_score(board, piece, square)
    return get_value(piece) + get_square_value(board, piece, square)
end

calc_score (generic function with 1 method)

## Ohne Inkrementelle Evaluation eines Zuges

In [8]:
function evaluate_position(board:: Board)
    if isterminal(board)
        return terminal_evaluation(board)
    end
    score = 0
    for x in 1:8,y in 1:8
        square = Square(SquareFile(x),SquareRank(y))
        piece = pieceon(board, square)
        if piece != EMPTY
            score += (Chess.pcolor(piece) == WHITE ? 1 : -1) * calc_score(board, piece, square)
        end
    end
    return score 
end

evaluate_position (generic function with 1 method)

## Inkrementelle Evaluation eines Zuges

### Capture Piece
The function `valueCapturePiece` is used to update the value of a chessboard when a piece is captured by the opponent. As input parameters it takes a chess board (board) and a chess piece that was captured (toMove).

When the function is called, the value of the captured piece is subtracted from the current value of the chess board.

In [9]:
function valueCapturePiece(board, square::Square)    
    capturePiece = pieceon(board, square)
    return capturePiece == EMPTY ? 0 : calc_score(board, capturePiece, square)
end

subCapturePiece (generic function with 1 method)

In [10]:
function checkEnPassant(board, toMove)
    enpassantSquare = epsquare(board)
    if(enpassantSquare != SQ_NONE && enpassantSquare == toMove)
        lastToMove = to(lastmove(board))
        return valueCapturePiece(board, lastToMove)
    end 
    return 0
end 

checkEnPassant (generic function with 1 method)

`calc_castle` checks wether the `move` done was a castleing move. 

If no the score will be returned back.

If yes it will adds the value of the move rook. The moved King will be calculated as the regularmove in the `evaluate_move` function. Casteling Kingside does not effect the value of the rook. Casteling Queenside adds +5 to the value of the position.

Check Castle KingSide is ignored, because the score is not changed. The reason is that the rook on square f1/f8 has the value zero. 

In [11]:
function calc_castle(board, piece, toMove)
    if(ptype(piece) == KING && cancastlequeenside(board, Chess.pcolor(piece)) && file(toMove) == FILE_C)
        return  5
    end 
    return 0
end

calc_castle (generic function with 1 method)

The function evaluate_move(board, move, score) takes in a `board` with it's current static centipawn `score` and a `move` and returns the static centipawn score after doing the move. The function is a incremental implementation meaning it will only consider and calculate the differences between the old and new position.

Following things need to be considered:
- Take away old square position value
- Add new square position value
- Take away score of captured pieces

Special events in chess
- castling
- promotion
- en passant
- entry into endgame

In [14]:
function evaluate_move(board:: Board, move:: Move, score)
    board_after_move = domove(board, move)
    if isterminal(board_after_move)
        return terminal_evaluation(board_after_move)
    end
    toMove = to(move)
    fromMove = from(move)
    piece = pieceon(board, from(move))
    pieceType = ptype(piece)
    color = Chess.pcolor(piece)
    
    # invert score if piece is black (imitate black is white)
    if color == BLACK
        score = -score
    end
    
    # Sub Points for old Position from Piece
    score -= valueCapturePiece(board, fromMove)
    score += valueCapturePiece(board, toMove)

    # en passant
    if pieceType == PAWN      
        score += checkEnPassant(board, toMove)
    end
    
    # promotion
    if ispromotion(move)      
        piece = Piece(sidetomove(board), promotion(move))
    end
    
    # castling
    if pieceType == KING     
        score += calc_castle(board, piece, toMove)
    end
    
    score += calc_score(board, piece, toMove)
    
    # invert back
    if color == BLACK
        score = -score
    end
    
    # entry into endgame
    if !is_endgame(board)
        if is_endgame(board_after_move)
            for kingPos in kings(board_after_move)
                k = pieceon(board_after_move, kingPos)
                score -= get_square_value(board, k, kingPos, "middle")
                score += get_square_value(board, k, kingPos, "end")
            end
        end
    end
    return score
end

evaluate_move (generic function with 1 method)

In [13]:
function terminal_evaluation(board:: Board)
    if ischeckmate(board)
        color = sidetomove(board)
        if color == WHITE
            return -100000
        else
            return 100000
        end
    elseif isstalemate(board)
        return 0
    elseif ismaterialdraw(board) # BAD
        return 0 
    elseif isrule50draw(board) 
        return 0
    end
    return
end

terminal_evaluation (generic function with 1 method)